In [ ]:
# 1. Compression by singular value decompositon:

import numpy as np
import matplotlib.pyplot as plt

matrix_size = [10,100,1500]
threshold = [1e-3,1e-6]

# Define the function T(x):
def T_1(x):
    T_function = np.exp(-0.4*np.tanh((x-7.7)/8))
    return T_function

# Define the function f(x,y):
def f_1(x,y):
    return (1/(np.sqrt(2*np.pi*T_1(x))))*np.exp(-(y**2)/2*T_1(x))
    return

# Second function:
def f_2(x,y):
    return -(x**2 + y**2) + 4

# Define the matrix F given an input size and a chosen function:
def matrix_F(f, m, n, x_1=0.1, x_2=14.5, y_1=-6, y_2=6):
    x = np.linspace(x_1,x_2,m)
    y = np.linspace(y_1,y_2,n)
    m_ = len(x)
    n_ = len(y)
    F = np.zeros([m_,n_])

    # Creating each element of the matrix using for-loops:
    for i in range(len(x)):
        for j in range(len(y)):
            F[i][j] = f(x[i],y[j])
    
    # Return F
    return F

# Calculate S via built-in SVD-function and plot the singular values,
# using a log-scale for the y-axis:
def SVD_plot_singular(f, m, n, x_1=0.1, x_2=14.5, y_1=-6, y_2=6):
    _,S,_ = np.linalg.svd(matrix_F(f,m,n,x_1,x_2,y_1,y_2))
    # Plot singular values:
    plt.figure()
    plt.plot(S, 'o')
    plt.xlabel('Index')
    plt.ylabel('Singular values')
    plt.yscale('log')
    plt.title(f"Singular values of F (n = {m})")

# Determining the rank r of truncated standard SVD for a given threshold:
def frobenius_singular_truncated(f,m,n,error,x_1=0.1,x_2=14.5,y_1=-6,y_2=6):
    _,S,_ = np.linalg.svd(matrix_F(f,m,n,x_1,x_2,y_1,y_2))
    frob_norm = np.sqrt(np.sum(S**2))
    for r in range(1,min(m,n)+1):
        trunc_error = np.sqrt(np.sum(S[r:]**2)) / frob_norm
        if trunc_error < error:
            return r, error, trunc_error
    return min(m,n), error, trunc_error

# Plot singular values for n = 10 and n = 100 of f(x,y):
for i in matrix_size:
    SVD_F = SVD_plot_singular(f_1,i,i)

# Find sufficient rank for to meet threshold criteria:
ranks_f1 = []
for p in [1,2]:
    print(f"\nMatrix F based on the given function (n = m = {matrix_size[p]}):")
    for i in range(len(threshold)):
        frob_trunc_r,frob_trunc_error,difference = frobenius_singular_truncated(f_1,matrix_size[p],matrix_size[p],threshold[i])
        ranks_f1.append(frob_trunc_r)
        print(f"For an error below {frob_trunc_error} a singular matrix of rank {frob_trunc_r} is sufficient.")

# Repeating the exercise for the second function:

# Use the already defined function for computing the singular values and plot them for n = 10 and n = 100:
for i in matrix_size:
    SVD_F = SVD_plot_singular(f_2,i,i,-2,2,-2,2)

# Find the sufficient rank needed for given threshold:
ranks_f2 = []
print(f"Matrix F based on first own function (n = m = {matrix_size[p]}):")
for n in range(len(threshold)):
    frob_trunc_r,frob_trunc_error,_ = frobenius_singular_truncated(f_2,matrix_size[p],matrix_size[p],threshold[n],-2,2,-2,2)
    ranks_f2.append(frob_trunc_r)
    print(f"For an error below {frob_trunc_error} a singular matrix of rank {frob_trunc_r} is sufficient.")

# 2. Randomized singular value decomposition
# 2.1: Implement the randomized range-finder algorithm and randomized SVD:

# Using Frobenius norm to estimate the necessary k for a given threshold:
def frob_norm_est(A,threshold=1e-3,k=1,step=1):
    m,n = A.shape
    frob_A = np.linalg.norm(A, ord='fro')
    for i in range(k,min(m,n),step):
        omega = np.random.normal(loc=0,scale=1,size=(n,i))
        Y = A @ omega
        Q,_ = np.linalg.qr(Y)
        A_k = Q @ (Q.T @ A)
        frob_difference = np.linalg.norm(A - A_k, ord='fro')
        error = frob_difference / frob_A
        if error < threshold:
            return i,error
    return np.min([m,n]), error

# Randomized SVD using the frob_norm_est function:
def rand_SVD_with_est(A, threshold=1e-3, k=1, step=1):
    k_new, error = frob_norm_est(A,threshold,k,step)
    m,n = A.shape
    omega = np.random.normal(loc=0, scale=1, size=(n,k_new))
    Y = A @ omega
    Q,_ = np.linalg.qr(Y)
    B = Q.T @ A
    U_wave,S_list,Vt = np.linalg.svd(B, full_matrices=False)
    U = Q @ U_wave
    return U,S_list,Vt,k_new,error

# Randomized SVD without using frob_norm_est function, instead simply computing for a given k:
def rand_SVD_without_est(A, k=5):
    m,n = A.shape
    omega = np.random.normal(loc=0, scale=1, size=(n,k))
    Y = A @ omega
    Q,_ = np.linalg.qr(Y)
    B = Q.T @ A
    U_wave,S_list,Vt = np.linalg.svd(B,full_matrices=False)
    U = Q @ U_wave
    return U,S_list,Vt

# 2.2 Redo experiments of Exercise 1
# Compare the truncated standard SVD to the RSVD

import time
import psutil
import os

k_begin = 1 # define an initial k value
functions = f_1,f_2
M_100 = matrix_F(f_1,100,100) # compute F for n = 100
M_5000 = matrix_F(f_1,5000,5000) # compute F for n = 5000

# Create lists for the respective singular values:
S_trunc = []
S_RSVD = []

# Plotting singular values now for RSVD for n = 10 and n = 100:
for f in functions:
    for n in matrix_size:
        plt.figure()
        plt.xlabel('Index')
        plt.ylabel('Singular values')
        plt.yscale('log')
        plt.title(f"Singular values of Matrix for {f.__name__} with (n = {n})")
        A = matrix_F(f,n,n)
        for k in range(n):
            _,S,_ = rand_SVD_without_est(A,k)
            plt.plot(S, 'o')

# Computing standard SVD and truncating for already computed ranks for matrix (100 x 100):
time_start_1 = time.time()
process_1 = psutil.Process(os.getpid())
memory_before_1 = process_1.memory_info().rss
_,S,_ = np.linalg.svd(M_100)
for i in ranks_f1:
    S_trunc.append(S[:(i)])
memory_after_1 = process_1.memory_info().rss
time_end_1 = time.time() - time_start_1
print(f"Computing singular values of matrix (100 x 100) of ranks 3 and 6:")
print(f"Takes {time_end_1:.8f} seconds and uses {memory_after_1 - memory_before_1} bytes for the standard SVD")

# Computing RSVD for already computed ranks for matrix (100 x 100):
time_start_2 = time.time()
process_2 = psutil.Process(os.getpid())
memory_before_2 = process_2.memory_info().rss
for i in ranks_f1:
    S_RSVD.append(rand_SVD_without_est(M_100,k=i)[1])
memory_after_2 = process_2.memory_info().rss
time_end_2 = time.time() - time_start_2
print(f"Takes {time_end_2:.8f} seconds and uses {memory_after_2 - memory_before_2} bytes for the RSVD\n")

# Computing standard SVD and truncating for already computed ranks for matrix (5000 x 5000):
time_start_3 = time.time()
process_3 = psutil.Process(os.getpid())
memory_before_3 = process_3.memory_info().rss
_,S,_ = np.linalg.svd(M_5000)
for i in ranks_f1:
    S_trunc.append(S[:(i)])
memory_after_3 = process_3.memory_info().rss
memory_used_3 = memory_after_3 - memory_before_3
time_end_3 = time.time() - time_start_3

print(f"Computing singular values of matrix (5000 x 5000) of ranks 3 and 6:")
print(f"Takes {time_end_3:.8f} seconds and uses {memory_used_3} bytes for the standard SVD")

# Computing RSVD for already computed ranks for matrix (100 x 100):
S_RSVD = []
time_start_4 = time.time()
process_4 = psutil.Process(os.getpid())
memory_before_4 = process_4.memory_info().rss
for i in ranks_f1:
    S_RSVD.append(rand_SVD_without_est(M_5000,k=i)[1])
memory_after_4 = process_4.memory_info().rss
memory_used_4 = memory_after_4 - memory_before_4
time_end_4 = time.time() - time_start_4

print(f"Takes {time_end_4:.8f} seconds and uses {memory_used_4} bytes for the RSVD")

# We compare the accuracy of the results using the Frobenius norm:
for i in [0,1]:
    frob_S = np.linalg.norm(S)
    frob_S_trunc = np.linalg.norm(S[:ranks_f1[i]]) / frob_S
    frob_S_RSVD = np.linalg.norm(S_RSVD[i]) / frob_S
    print()

    print(f"Accuracy of truncated and randomized SVD's:")
    print(f"Accuracy of truncated S with rank {ranks_f1[i]}: {frob_S_trunc*100} %")
    print(f"Accuracy of S from RSVD with rank {ranks_f1[i]}: {frob_S_RSVD*100} %")

# If-statement, in the case a memory-use of 0 bytes is measured:
if memory_used_4 == 0:
    print(f"\nFor a matrix with dimensions 5000 x 5000, RSVD is approximately\n- {(time_end_3 / time_end_4):.0f} times faster\n- Uses no memory as compared to {memory_used_3} bytes for the standard SVD")
else:
    print(f"\nFor a matrix with dimensions 5000 x 5000, RSVD is approximately\n- {(time_end_3 / time_end_4):.0f} times faster\n- uses 1/{(memory_used_3 / memory_used_4):.0f}th of the memory")

# 3. Image compression:

import time
import matplotlib.pyplot as plt
from skimage.color import rgb2gray

# Load image to be used:
image = plt.imread('swimming_with_turtles_dudok_de_witt.jpg')

# Convert image to gray scale, if RGB:
if image.ndim == 3:  # If the image has 3 channels (RGB)
    image_gray = rgb2gray(image)
else:
    image_gray = image  # If the image is already grayscale

B = image_gray # B is a matrix and reference for the gray scale image
tolerances = np.round(np.arange(0.1, 0.0099, -0.01),2) # Set up range of tolerances to be used
k = 1 # Initial k value

# Create lists to store results to be used for the plots:
runtime_RSVD = []
runtime_standard = []
k_list = []

# Find the k's for the given tolerances from above and save in list:
for n in range(len(tolerances)):
    U,S_list,Vt,k_new, _ = rand_SVD_with_est(B,tolerances[n],k)
    S = np.diag(S_list)
    B_k = U @ (S @ Vt)
    k_list.append(k_new)

# Stanard SVD computation with already computed k values, and creating list for the runtimes:
time_start_1 = time.time()
U,S_list,Vt = np.linalg.svd(B,full_matrices=False)
for n in k_list:
    size = np.arange(0,n)
    B_k = U[:,size] @ np.diag(S_list[size]) @ Vt[size,:]
    time_finish_1 = time.time() - time_start_1
    runtime_standard.append(time_finish_1)

total_time_standard = np.sum(runtime_standard)

# RSVD computation with already computed k values, and creating list for the runtimes times:
for n in k_list:
    time_start_0 = time.time()
    U,S_list,Vt = rand_SVD_without_est(B,n)
    S = np.diag(S_list)
    B_k = U @ (S @ Vt)
    time_end_0 = time.time() - time_start_0
    runtime_RSVD.append(time_end_0)

total_time_RSVD = np.sum(runtime_RSVD)

print(f"\nTotal time for the standard SVD is {total_time_standard:.3f} seconds, while the total runtime of RSVD is {total_time_RSVD:.3f} seconds.")

print(f"\nThe values for k are:\n{k_list}")

# Plot the tolerances vs. CPU time for the two methods
plt.figure()
plt.plot(tolerances, runtime_RSVD, 'o-')
plt.plot(tolerances, runtime_standard, 'o-')
plt.title('Tolerances & CPU Time')
plt.xlabel('Tolerances')
plt.ylabel('CPU Time')
plt.gca().invert_xaxis()
plt.yscale('log')
plt.grid()
plt.show()

# Showing the image
plt.figure(dpi=300)
plt.title("Image from The Red Turtle")
plt.imshow(image);



Matrix F based on the given function (n = m = 100):
For an error below 0.001 a singular matrix of rank 3 is sufficient.
For an error below 1e-06 a singular matrix of rank 6 is sufficient.

Matrix F based on the given function (n = m = 1500):
For an error below 0.001 a singular matrix of rank 3 is sufficient.
For an error below 1e-06 a singular matrix of rank 6 is sufficient.
Matrix F based on first own function (n = m = 1500):
For an error below 0.001 a singular matrix of rank 2 is sufficient.
For an error below 1e-06 a singular matrix of rank 2 is sufficient.
